In [1]:
import pandas as pd
import bayes 
import json


In [6]:
df = pd.read_csv(r"C:\Users\Muc\Downloads\CGa_0111_0811 (1).csv")
print(df.app_version.value_counts())
print(df.experiment_group.value_counts())



1.1.6    167278
1.1.3      1380
1.1.2       563
1.1.1        85
1.1.0         5
1.0.1         2
Name: app_version, dtype: int64
firebase_exp_8_group1    26571
firebase_exp_8_group0    26541
firebase_exp_8_group2    24597
Name: experiment_group, dtype: int64


In [7]:
from datetime import timedelta

def preprocess(data, metrics, ab_list, compare_by,  day_window = 3, **kwargs):
    data.loc[:,'day0_date'] = pd.to_datetime(data.loc[:,'day0_date'])
    data.loc[:,'act_date'] = pd.to_datetime(data.loc[:,'act_date'])
    if (data.act_date.max() - data.day0_date.min()) > timedelta(3):
        data = data[data.loc[:,'act_date'] < data.loc[:,'act_date'].max()] #drop max act_date 
        data = data[data.loc[:,'day0_date'] <= data.loc[:,'act_date'].max()-timedelta(days=day_window)] #find range day0_date
        data = data[(data.loc[:,'day_diff'] <= day_window) & (data.loc[:,'day_diff']>= 0)]

    for key, value in kwargs.items():
        data = data[data.loc[:,key] == value]
    
    if compare_by == 'app_version':
        data = data.query('app_version in @ab_list')
    elif compare_by == 'experiment_group':
        data = data.query('experiment_group in @ab_list')
    
    data['combined_ad'] = data['rv_imp_sum'] * 1.3 + data['is_imp_sum']
    dim = data.groupby('resettable_device_id_or_app_instance_id')[compare_by].max().reset_index()
    fact = data.groupby('resettable_device_id_or_app_instance_id')[metrics].sum()
    data = dim.merge(fact, on ='resettable_device_id_or_app_instance_id')
    return data


In [57]:
metrics = [
    'time_in_game_sum'
    ,'rv_imp_sum'
    ,'is_imp_sum'
    ,'combined_ad'
    ,'battle_play_sum'
    ,'battle_end_sum'
    # ,'ad_value_sum'
]

filter = {
    'country':'Vietnam'
    ,'event_segment': 'app_update--FALSE'
    ,'app_version' : '1.1.6'
    # ,'media_source' : "Organic"
}
# ab_list = ['1.1.2', '1.1.3']
ab_list = ['firebase_exp_8_group0', 'firebase_exp_8_group2']


data = preprocess(df,metrics,ab_list, compare_by = 'experiment_group', day_window = 3, **filter)
descriptive_metric = []
# number_user = data.groupby('app_version').agg({'resettable_device_id_or_app_instance_id':pd.Series.nunique})
number_user = data.groupby('experiment_group').agg({'resettable_device_id_or_app_instance_id':pd.Series.nunique})



for i in metrics:
    descriptive_metric.append(data.groupby('experiment_group').agg({i: ['mean', 'std']}))
descriptive_metric = pd.concat(descriptive_metric, axis=1)
descriptive_metric
for metric in metrics:
    data_metrics = descriptive_metric[metric]
    na, nb = number_user.resettable_device_id_or_app_instance_id
    ma, mb = data_metrics['mean']
    sa, sb = data_metrics['std']
    globals()[metric] = bayes.gaussian_ab_test(m_a=ma, s_a=sa, n_a=na, m_b=mb, s_b=sb, n_b=nb)

C:\Users\Muc\AppData\Local\Temp/ipykernel_12040/688019483.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['combined_ad'] = data['rv_imp_sum'] * 1.3 + data['is_imp_sum']


In [30]:
na

783

In [45]:
time_in_game_sum

{'chance_to_win': 0.0009254425218664757,
 'expected': -0.23934395931628583,
 'ci': [-0.35969356024106225, -0.09637389802536689],
 'uplift': {'dist': 'lognormal',
  'mean': -0.2735740066285395,
  'stddev': 0.08787630784496527},
 'risk': [0.0054833872260360295, 59.167357512542],
 'mean': {'m_a': 247.18348561759728, 'm_b': 188.0216114922813}}

In [58]:
rv_imp_sum

{'chance_to_win': 1.2456315888632505e-06,
 'expected': -0.2927427485064662,
 'ci': [-0.38769621009223487, -0.18306430886939695],
 'uplift': {'dist': 'lognormal',
  'mean': -0.3463608157692,
  'stddev': 0.07355538992539119},
 'risk': [5.2776494730011336e-08, 0.6308086656749459],
 'mean': {'m_a': 2.1548223350253806, 'm_b': 1.5240137221269296}}

In [59]:
is_imp_sum

{'chance_to_win': 3.7685669424098512e-06,
 'expected': -0.2855563353689373,
 'ci': [-0.38333317700751735, -0.17227629102127862],
 'uplift': {'dist': 'lognormal',
  'mean': -0.3362511305834668,
  'stddev': 0.07509080098104765},
 'risk': [6.382264672311067e-07, 2.561310509276784],
 'mean': {'m_a': 8.969543147208121, 'm_b': 6.408233276157804}}

In [61]:
battle_end_sum

{'chance_to_win': 2.3480304396554294e-05,
 'expected': -0.2819358471839457,
 'ci': [-0.38778899940282907, -0.1577803615804213],
 'uplift': {'dist': 'lognormal',
  'mean': -0.3311963645993452,
  'stddev': 0.0813698212472897},
 'risk': [7.563632239637741e-06, 3.6704211139423517],
 'mean': {'m_a': 13.018612521150592, 'm_b': 9.34819897084048}}

In [62]:
combined_ad

{'chance_to_win': 4.6883234213261187e-07,
 'expected': -0.2872665891973203,
 'ci': [-0.3774840398721536, -0.1839744722848624],
 'uplift': {'dist': 'lognormal',
  'mean': -0.3386478263775414,
  'stddev': 0.06905136364868851},
 'risk': [1.0026150931707889e-07, 3.381361168079815],
 'mean': {'m_a': 11.770812182741118, 'm_b': 8.389451114922814}}

In [60]:
battle_play_sum

{'chance_to_win': 0.00011039626184711519,
 'expected': -0.23588872652231707,
 'ci': [-0.3375378871656931, -0.11864236921014493],
 'uplift': {'dist': 'lognormal',
  'mean': -0.2690418545077553,
  'stddev': 0.07283299976424865},
 'risk': [3.4877990017889715e-05, 3.4559444187573316],
 'mean': {'m_a': 14.650592216582064, 'm_b': 11.194682675814752}}